In [5]:
import numpy as np
from micrograd.engine import Value
from typing import List


In [11]:

class Neuron:


    def __init__(self, ins):
        self.w = [Value(np.random.uniform(-1, 1)) for _ in range(ins)]
        self.b = Value(np.random.uniform(-1, 1))


    def eval(self, args):
        product = sum(wi * xi for wi, xi in zip(self.w, args))
        return (product + self.b).relu()
    
    def evalo(self, args):
        product = sum(wi * xi for wi, xi in zip(self.w, args))
        return (product + self.b)
    
    def update(self, learn_r):
        for i in range(len(self.w)):
            self.w[i].data = self.w[i].data - learn_r*self.w[i].grad
        self.b.data = self.b.data - learn_r*(self.b.grad)
    def reset_grad(self):
        for i in range(len(self.w)):
            self.w[i].grad = 0.0
        self.b.grad = 0.0

        

class Network:
    def __init__(self):
        self.h1 = [Neuron(3) for _ in range(4)]
        self.h2 = [Neuron(4) for _ in range(2)]
        self.o = Neuron(2)
    
    def outputf(self, inputs: List):
        a1 = [neuron.eval(inputs) for neuron in self.h1]
        a2 = [neuron.eval(a1) for neuron in self.h2]
        output = self.o.evalo(a2)

        return output
    def mse_loss(self, output, y):
            loss = 0.5*(output-y)**2
            return loss
    


    
    def sgd_training(self, epochs, lr, train_set):
        
        
        def backprop():
            self.o.update(lr)
              
            for neuron in self.h2:
                    neuron.update(lr)
                    
            for neuron in self.h1:
                    neuron.update(lr)

            self.o.reset_grad()
              
            for neuron in self.h2:
                    neuron.reset_grad()
                    
            for neuron in self.h1:
                    neuron.reset_grad()
        
        for epoch in range(epochs):
            epoch_loss = 0
            for train_ex in train_set:
                ex_loss = self.mse_loss(self.outputf(train_ex[0]), train_ex[1])
                epoch_loss += ex_loss.data
                ex_loss.backward()
                backprop()
            
            avg_epoch_loss = epoch_loss/len(train_set)
            print(f"Average Loss per Epoch: {avg_epoch_loss}")

    def testing(self, test_set):
        _loss = 0
        for test_ex in test_set:
            ex_loss = self.mse_loss(self.outputf(test_ex[0]), test_ex[1])
            _loss += ex_loss.data
        avg_loss = _loss/len(test_set)
        print(f"Average Loss per Epoch: {avg_loss}")
         

            
            

In [15]:
import numpy as np
import math
from typing import List # Assuming Value class is defined above

# =============================================================================
# IMPORTANT: Ensure the Value class definition exists BEFORE this code runs!
# =============================================================================
# class Value:
#    ... (full Value class definition here or imported) ...
# =============================================================================


# --- Dataset Generation ---

# Define the underlying function (without noise)
def target_function(x1, x2, x3):
    """The true function we want the network to approximate."""
    term1 = 1.5 * math.sin(math.pi * x1)
    term2 = 0.8 * math.cos(math.pi * x2) * x3
    term3 = 0.1 * x1 * x2 * x3
    return term1 + term2 + term3


def generate_dataset_with_values(num_samples, noise_std_dev=0.15):

    dataset = []
    # Generate raw input floats uniformly between -1 and 1
    raw_inputs = np.random.uniform(-1, 1, size=(num_samples, 3))

    for i in range(num_samples):
        x1, x2, x3 = raw_inputs[i] # Get raw floats

        # Calculate the true output using the target function (still using floats)
        y_true = target_function(x1, x2, x3)

        # Add Gaussian noise (still using floats)
        noise = np.random.normal(loc=0.0, scale=noise_std_dev)
        y_noisy_float = y_true + noise

        # --- Wrap the floats as Value objects ---
        input_list_val = [Value(x1), Value(x2), Value(x3)]
        target_val = Value(y_noisy_float)

        # Store the tuple containing Value objects
        dataset.append((input_list_val, target_val))

    return dataset

# --- Generate the Datasets with Value Objects ---

# Training Dataset
num_train_samples = 500
train_set_val = generate_dataset_with_values(num_train_samples, noise_std_dev=0.15)

# Testing Dataset
num_test_samples = 100
test_set_val = generate_dataset_with_values(num_test_samples, noise_std_dev=0.15)
  

In [17]:
print(train_set_val[0:10])

[([Value(data=-0.13469297053419438, grad=0), Value(data=0.12307533427942263, grad=0), Value(data=-0.7255367870503906, grad=0)], Value(data=-0.8873922490585191, grad=0)), ([Value(data=-0.24225122165405977, grad=0), Value(data=0.8930840115940133, grad=0), Value(data=0.787925378366017, grad=0)], Value(data=-1.5854866824827476, grad=0)), ([Value(data=-0.09647805859314418, grad=0), Value(data=0.5391532561600321, grad=0), Value(data=-0.9249626100815938, grad=0)], Value(data=-0.2685336393770528, grad=0)), ([Value(data=0.11062946551533304, grad=0), Value(data=0.12330967909241664, grad=0), Value(data=-0.350412084955003, grad=0)], Value(data=0.04218420561159819, grad=0)), ([Value(data=-0.8317062483884143, grad=0), Value(data=0.46448155393525026, grad=0), Value(data=-0.7491277843769826, grad=0)], Value(data=-0.7659169347899284, grad=0)), ([Value(data=0.4624059009031882, grad=0), Value(data=0.08283751289893915, grad=0), Value(data=-0.21228921961020197, grad=0)], Value(data=1.4501074633046194, grad

In [19]:
net = Network()
epochs = 120
learning_rate = 0.005
print("\n--- Starting Training ---")
 # Pass the datasets containing Value objects directly
net.sgd_training(epochs, learning_rate, train_set_val)
print("--- Training Finished ---")

print("\n--- Evaluating on Test Set ---")
net.testing(test_set_val)
print("--- Evaluation Finished ---")



--- Starting Training ---
Average Loss per Epoch: 0.6341106787071962
Average Loss per Epoch: 0.5894551440373429
Average Loss per Epoch: 0.5662250196916495
Average Loss per Epoch: 0.527146755222293
Average Loss per Epoch: 0.4637378617593702
Average Loss per Epoch: 0.3732853955438436
Average Loss per Epoch: 0.2919248686202846
Average Loss per Epoch: 0.2525035264664222
Average Loss per Epoch: 0.23341779384309372
Average Loss per Epoch: 0.22055895387022076
Average Loss per Epoch: 0.21065403751084205
Average Loss per Epoch: 0.20117266433907463
Average Loss per Epoch: 0.1920602410290171
Average Loss per Epoch: 0.1830804613011729
Average Loss per Epoch: 0.1738183304133305
Average Loss per Epoch: 0.1641381780797741
Average Loss per Epoch: 0.15347062816424706
Average Loss per Epoch: 0.14314161578976375
Average Loss per Epoch: 0.1339917009012564
Average Loss per Epoch: 0.12623287856777346
Average Loss per Epoch: 0.11982801072700155
Average Loss per Epoch: 0.11477114605847653
Average Loss per Ep

In [21]:
test_instance1 = [[Value(0.03), Value(0.02), Value(0.01)], Value(target_function(0.03,0.02,0.01))]

print(f"Predicted Output: {net.outputf(test_instance1[0])}")
print(f"True Output: {test_instance1[1]}")
print(f"Loss: {net.mse_loss(net.outputf(test_instance1[0]), test_instance1[1])}")

Predicted Output: Value(data=0.042819160992059846, grad=0)
True Output: Value(data=0.14914728380519762, grad=0)
Loss: Value(data=0.005652834850482855, grad=0)


In [27]:

print("--- Generating Noise-Free Datasets ---")
num_train_samples = 500 # Keep size consistent
num_test_samples = 100

# Generate datasets with NO noise
train_set_zero_noise = generate_dataset_with_values(num_train_samples, noise_std_dev=0.0)
test_set_zero_noise = generate_dataset_with_values(num_test_samples, noise_std_dev=0.0)
print("--- Noise-Free Datasets Generated ---")


# --- 2. Initialize and Train net2 ---

print("\n--- Initializing and Training net2 (Noise-Free Data) ---")
net2 = Network() # Create a new, independent network instance

# Use the same hyperparameters as before (assuming these were the last successful ones)
epochs = 1200
learning_rate = 0.0005

print("\n--- Starting Training ---")
 # Pass the datasets containing Value objects directly
net.sgd_training(epochs, learning_rate, train_set_val)
print("--- Training Finished ---")

print("\n--- Evaluating on Test Set ---")
net.testing(test_set_val)
print("--- Evaluation Finished ---")

# --- 3. Repeat the Experiment with net2 ---

print("\n--- Testing net2 on Specific Instance [0.03, 0.02, 0.01] ---")

# Define the specific test instance inputs (same as before)
inputs_float = [0.03, 0.02, 0.01]
inputs_val = [Value(x) for x in inputs_float]

# Calculate the true target (it's deterministic, so same as before)
true_target_float = target_function(inputs_float[0], inputs_float[1], inputs_float[2])
true_target_val = Value(true_target_float)

# Get prediction from net2
predicted_output_net2 = net2.outputf(inputs_val)

# Calculate loss using net2's prediction
loss_net2 = net2.mse_loss(predicted_output_net2, true_target_val)

# Print results for net2
print(f"Inputs: {inputs_float}")
print(f"Predicted Output (net2): {predicted_output_net2}")
print(f"True Output            : {true_target_val}")
print(f"Loss (net2)            : {loss_net2}")

# --- Optional: Compare with previous net results if desired ---
# print("\n--- For Comparison (Original net on Noisy Data) ---")
# Assuming 'net' still exists and holds the state trained on noisy data
# predicted_output_net1 = net.outputf(inputs_val)
# loss_net1 = net.mse_loss(predicted_output_net1, true_target_val)
# print(f"Predicted Output (net1): {predicted_output_net1}")
# print(f"Loss (net1)            : {loss_net1}")

--- Generating Noise-Free Datasets ---
--- Noise-Free Datasets Generated ---

--- Initializing and Training net2 (Noise-Free Data) ---

--- Starting Training ---
Average Loss per Epoch: 0.055110349067181645
Average Loss per Epoch: 0.05502619868204296
Average Loss per Epoch: 0.05496942682974668
Average Loss per Epoch: 0.05493342900161474
Average Loss per Epoch: 0.054909085268858966
Average Loss per Epoch: 0.05490384602210619
Average Loss per Epoch: 0.05488148457226414
Average Loss per Epoch: 0.05486960652046483
Average Loss per Epoch: 0.05486130935163641
Average Loss per Epoch: 0.054855363444645346
Average Loss per Epoch: 0.05485077672249431
Average Loss per Epoch: 0.05484716914381711
Average Loss per Epoch: 0.054844264541004654
Average Loss per Epoch: 0.054841875990149946
Average Loss per Epoch: 0.05483987453962452
Average Loss per Epoch: 0.05483816914916512
Average Loss per Epoch: 0.05483669414203995
Average Loss per Epoch: 0.05483540118291353
Average Loss per Epoch: 0.054834254039040